In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
with open("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/integrated_network.pickle","rb") as file:
    network = pickle.load(file)

In [ ]:
with open("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/integrated_df.pickle","wb") as file:
    pickle.dump(df,file)

In [ ]:
df = pd.DataFrame.from_dict(dict(network.nodes(data=True)), orient='index') 

In [ ]:
plastchem =  pd.read_csv("/lustre/BIF/nobackup/hendr218/Databases/PlastChem_db.tsv", sep='\t', encoding='windows-1251', low_memory=False, header=1)

In [ ]:
test = plastchem["inchi"][plastchem["inchi"].notna()]

In [ ]:
test.to_csv("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/plastchem_inchi_commasep.csv",header=False,index=False,sep=",")

In [ ]:
test = plastchem["molecular_formula"][plastchem["molecular_formula"].notna()]

In [ ]:
test = plastchem["canonical_smiles"][plastchem["canonical_smiles"].notna()]

In [ ]:
test.to_csv("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/plastchem_smiles.csv",header=False,index=False)

In [ ]:
test.to_csv("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/plastchem_smiles_commasep.csv",header=False,index=False,sep=",")

In [ ]:
quantt =  pd.read_csv("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/V2_test1_pos/mzmine/mzmine_iimn_fbmn_quant_full.csv"

In [ ]:
quantt[]


In [ ]:
samples = [key for key in quantt.keys() if ("mzML:area" in key) & (not "Blank" in key)]
blanks = [key for key in quantt.keys() if ("mzML:area" in key) & ("Blank" in key)]

In [ ]:
sample_areas = quantt.loc[:,samples].transpose()
blank_areas =quantt.loc[:,blanks].transpose()

In [ ]:
sample_means = sample_areas.mean().to_frame()
blank_means = blank_areas.mean().to_frame()

In [ ]:
blank_means["sample_means"] = sample_means 

In [ ]:
blank_means["sample_means"] = sample_areas.mean()

In [ ]:
means = blank_means.rename(columns={0:"blank_means"})

In [ ]:
means = means.fillna(0)

In [ ]:
means_checked = means["blank_means"] < 0.3*means["sample_means"]

In [ ]:
means_checked.to_dict()

In [ ]:
test2 = sample_areas.transpose()


In [ ]:
test2["mean"] = test2.mean(axis=1)

In [ ]:
test2_sample = [key for key in test2.keys() if "PE_PET_PC" in key]

In [ ]:
test2 = test2.transpose().fillna(0)

In [ ]:
test2

In [ ]:
for id in test2.keys():
    mean = test2[id].mean()
    test3[id] = test3[id].map(lambda x: True if mean < 0.3 * x else False)
    # test[test["mean"] < 0.3*test.loc[:,test_sample].mean(axis=1)]
    # lambda x: True if x > mean else False




In [ ]:
test3 = test2

In [ ]:
test3

In [ ]:
test4 = {}
for id, files in test3.to_dict().items():
    test4[id] = ""
    for file,val in files.items(): 
        if (val) & (not file in test4[id]):
            test4[id] += file[:-5]

In [ ]:
test4

In [ ]:
test3.to_dict().keys()

In [ ]:
test3_dict

In [ ]:
test[test["mean"] < 0.3*test.loc[:,test_sample].mean(axis=1)]

In [ ]:
means_copy = test

In [ ]:
means_copy

In [ ]:
with open("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/V2_test1_pos/full_network_df.pickle","rb") as file:
    dataframe = pickle.load(file)

In [ ]:
"""takes input mgf,df and adds metadata"""
with open("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/V2_test1_pos/mzmine/mzmine_sirius.mgf") as file:
    A = file.read()
    entries = A.split("BEGIN IONS")[1:]

id_files_dict = {}
for entry in entries:
    feature_id = entry.split('\n')[1][11:] 
    if "FILENAME" in entry:
        files = entry.split("FILENAME")[1][1:].split('\n')[0]
        id_files_dict[feature_id] = files.split(";")

metadata_dict = {}
with open("/lustre/BIF/nobackup/hendr218/Data/testrun_06_02/input/metadata_pos_17-02.csv") as file:
    for line in file.readlines()[1:]:
        name,plastic_type,plastic_state = line.split(',')
        plastic_state=plastic_state[:-1]
        polarity = "pos" if "pos" in name else "neg"
        if plastic_state == "Blank":
            state_type = "blank"
        elif plastic_state == "procBlank":
            state_type = "proc_blank"
        else:
            state_type = f"{plastic_type}_{plastic_state}"
        if not name in metadata_dict:
            metadata_dict[name] = state_type
        


id_statetype = {}
test = id_files_dict["4"]
for id, files in id_files_dict.items():
    id_statetype[id] = ""
    for file in files: 
        statetype = metadata_dict[file]
        if not statetype in id_statetype[id]:
            id_statetype[id] += f"{statetype}, "

#  for i in id_statetype["id"].unique():
dataframe["all_metadata"] = dataframe.index.map(lambda x: id_statetype.get(x,"N/A"))
for metadata_type in ["PE_PA_Dec", "PE_PA_PC","PE_Dec", "PE_PC", "PE_PET_PC", "PE_PET_Dec","blank","PE_PA","PE_PET"]:
    dataframe[metadata_type] = dataframe["all_metadata"].str.contains(metadata_type)
dataframe["PE"] = dataframe["all_metadata"].map(lambda x: True if ("PE_Dec" in x or "PE_PC" in x) else False)



In [ ]:
dataframe["all_metadata"] = dataframe.index.map(lambda x: test4.get(x,"N/A"))
for metadata_type in ["PE_PA_Dec", "PE_PA_PC","PE_Dec", "PE_PC", "PE_PET_PC", "PE_PET_Dec","blank","PE_PA","PE_PET"]:
    dataframe[metadata_type] = dataframe["all_metadata"].str.contains(metadata_type)
dataframe["PE"] = dataframe["all_metadata"].map(lambda x: True if ("PE_Dec" in x or "PE_PC" in x) else False)

In [ ]:
test4

In [ ]:
dataframe.loc[["1"],:]

In [ ]:
id_statetype

In [ ]:
metadata_dict


In [ ]:
quant_table =  pd.read_csv("/lustre/BIF/nobackup/hendr218/Data/plastchem_as_sirius_db/all/mzmine/mzmine_iimn_fbmn_quant_full.csv",low_memory=False)
samples = [key for key in quant_table.keys() if ("mzML:area" in key) & (not "Blank" in key)]
samples.append('id')
blanks = [key for key in quant_table.keys() if ("mzML:area" in key) & ("Blank" in key)]
blanks.append('id')
s_areas = quant_table.loc[:,samples].set_index('id').transpose()
b_areas =quant_table.loc[:,blanks].set_index('id').transpose()

s_area_tresholded = s_areas
for id in s_areas.keys():
    mean = s_areas[id].mean()
    s_area_tresholded[id] = s_areas[id].map(lambda x: True if mean < 0.3 * x else False)

test4 = {}
for id, files in s_area_tresholded.to_dict().items():
    id = str(id)
    test4[id] = ""
    for file,val in files.items(): 
        pass
        if (val) & (not file in test4[id]):
            test4[id] += f"{file[:-5]}, "

In [ ]:
b_areas = quant_table.loc[:,blanks].set_index('id').transpose().fillna(0)

In [ ]:
test1 = quant_table.loc[:,samples].set_index('id').transpose().fillna(0).mean()

In [ ]:
test2 = quant_table.loc[:,blanks].set_index('id').transpose().fillna(0).max()

In [ ]:
test1[test1 > 10*test2]

In [ ]:
b_areas

In [ ]:
quant_table =  pd.read_csv("/lustre/BIF/nobackup/hendr218/Data/plastchem_as_sirius_db/all/mzmine/mzmine_iimn_fbmn_quant_full.csv",low_memory=False)
samples = [key for key in quant_table.keys() if ("mzML:area" in key) & (not "Blank" in key)]
samples.append('id')
blanks = [key for key in quant_table.keys() if ("mzML:area" in key) & ("Blank" in key)]
blanks.append('id')
s_areas = quant_table.loc[:,samples].set_index('id').transpose()
b_areas =quant_table.loc[:,blanks].set_index('id').transpose()

s_area_tresholded = s_areas
for id in s_areas.keys():
    mean = s_areas[id].mean()
    blank_mean = b_areas[id].mean()


test5= {}
for id, files in s_area_tresholded.to_dict().items():
    id = str(id)
    test5[id] = ""
    for file,val in files.items(): 
        pass
        if (val) & (not file in test5[id]):
            test4[id] += f"{file[:-5]}, "

In [ ]:
test = pd.DataFrame(index=s_area_tresholded.keys())

In [ ]:
test

In [ ]:
test["is_blank"] = range(len(test))

In [ ]:
test.loc[len(test)] = pd.Series(dtype='float64')

In [ ]:
blank_mean

In [ ]:
for id in s_areas.keys():
    mean = quant_table.loc[:,samples].set_index('id').transpose().fillna(0).mean()
    blank_max = quant_table.loc[:,blanks].set_index('id').transpose().fillna(0).max()
test["is_blank"] = test.index.map(lambda x: False if mean > 0.3*blank_max else True)


In [ ]:
b_areas = b_areas.fillna(0)

In [ ]:
b_areas[4].mean()

In [ ]:
test[test["is_blank"] == True]

In [ ]:
test["is_blank"] = test.index.map(lambda x: )

In [ ]:
dataframe["all_metadata"] = dataframe.index.map(lambda x: test5.get(x,"N/A"))
for metadata_type in ["PE_PA_Dec", "PE_PA_PC","PE_Dec", "PE_PC", "PE_PET_PC", "PE_PET_Dec","blank","PE_PA","PE_PET"]:
    dataframe[metadata_type] = dataframe["all_metadata"].str.contains(metadata_type)
dataframe["PE"] = dataframe["all_metadata"].map(lambda x: True if ("PE_Dec" in x or "PE_PC" in x) else False)

In [ ]:
ndataframe.index

In [ ]:
import pickle

In [ ]:
class WorkflowRunner():
    '''Holds the full workflow

    Used to  use the workflow for a specific combination of parameters and settings. 

    Parameters
    ----------
    settings : str | dict
        fn of settings file or dict in json format
        should contain settings defined in workflowsettings required settings class property     

    Attributes
    ----------
    available_tools : list
        Defines which tools can be run. runtool should also be set True in settings file to actually run the tool.
    available_integrations : list
        Defines which integrations can be selected. Integrate_tool should also be set True in settings file to actually integrate the tool.

    '''
    
    available_tools: list[str] = ["mzmine","ms2lda","sirius","toxtree","classyfire", "matchms"]
    available_integrations: list[str] = ["mzmine","ms2lda","sirius","toxtree","classyfire", "matchms","plastchemdb","sirius_db"]

    def __init__(self,settings_json: str | dict) -> None:
        self.settings: WorkflowSettings = WorkflowSettings(settings_json,available_tools=WorkflowRunner.available_tools,available_integrations=WorkflowRunner.available_integrations)
        self.name = self.settings.name
        self.output_folder = self.settings.output_folder
        self.graph: nx.Graph = nx.Graph
        self.network_df, self.network_edgelist = pd.DataFrame,nx.edgelist

    def get_internal_settings(self,settings_json: str | dict) -> dict:
        """Check if internal settings were provided and return them as a json dict
        
        Parameters
        ----------
        settings_json : str | dict
            fn of settings file or settings dict in json format

        Returns
        -------
        Settings dict in json format
        """
        try:
            fn = json_from_string_or_file(settings_json)["paths"].get("internal_settings")
        except KeyError:
            print("settings file must contain 'internal_setting' path in paths")
        return json.load(fn)
    
    def run_all(self) -> None:
        self.run_and_integrate_per_depth()
        print("Sucesfully ran and integrated all tools")

    def get_depths(self,tools: Iterable,goal:str) -> dict[Any,str]:
        """"""
        depths: dict = {}
        for tool in tools:
            config = self.settings.config.get(tool,{})
            goal_depth = config.get(goal,{}).get("depth")
            depth = goal_depth if goal_depth else config.get("depth","N/A")
            match depth:
                case  "1" | 1:
                    depths[tool] = 1
                case  "2" | 2:
                    depths[tool] = 2
                case  "3" | 3:
                    depths[tool] = 3
                case  "4" | 4:
                    depths[tool] = 4
                case  "N/A":
                    depths[tool] = "N/A"
        return depths

    def run_and_integrate_per_depth(self) -> None:
        """For every depth in ascending order, run and integrate tools that are of that depth, finally produce the integrated network graphml file.

        Every tool should have a depth for running and integration specified in internal settings. For every depth, first the tools will be ran, then integrated. 
        This is only attempted for tools that were selected in class property `available tools` or `available integrations`. 
        As the first thing on depth 3, the graph is made so any tools requiring the graph should be of depth 3+.
        As the first thing on depth 4, the network is made so any tools requiring the network should be of depth 4+.

        """

        running_depths: dict[Any,str] = self.get_depths(self.settings.to_run,goal="running")
        integration_depths: dict[Any,str] = self.get_depths(self.settings.to_integrate,goal="integration")
        for current_depth in [1,2,3,4,"N/A"]:
            running: set[Any] = {tool for tool,depth in running_depths.items() if depth==current_depth}
            integrating: set[Any] = {tool for tool,depth in integration_depths.items() if depth==current_depth}               
            if current_depth == 3:
                self.graph = nx.read_graphml(self.settings.paths.get("base_network"))
            if current_depth == 4:   
                self.network_df, self.network_edgelist = network_to_edgelist_and_nodes_df(self.graph)
                self.integration_col = "smiles"  
                #Temporary solution: needs consensus column for smiles/inchis
                self.network_df["smiles"] = self.network_df.apply(self.get_consensus_smiles,axis=1)
                self.network_df["Molecular formula"] = self.network_df.apply(self.get_consensus_formula,axis=1)
            for tool in running:
                self.run_tool(tool)
            for integration in integrating:
                self.integrate_tool(integration)
       # target_df[target_df["Molecular formula"].notna()].merge(source_df[source_df["molecular_formula"].notna()],left_on="Molecular formula",right_on="molecular_formula")
        self.produce_integrated_graphml()

    def get_consensus_formula(self,row):
        """Get formula from database sirius formula if exists, otherwise normal formula"""
        # if row["library_smiles"]
        if not row.get("sirius:molecularFormula"): 
            row["sirius:molecularFormula"] = "N/A"
        if row.get("sirius_db:molecularFormula"):
            return row["sirius_db:molecularFormula"] if isinstance(row["sirius_db:molecularFormula"],str) else row["sirius:molecularFormula"]
        else:
            return row["sirius:molecularFormula"] 
        
    def get_consensus_smiles(self,row):
        """Get smiles from database sirius smiles if exists, otherwise normal smiles"""
        # if row["library_smiles"]
        if not row.get("csifingerid:smiles"): 
            row["csifingerid:smiles"] = "N/A"
        if row.get("sirius_db:molecularFormula"):
            return row["csifingerid_db:smiles"] if isinstance(row["csifingerid_db:smiles"],str) else row["csifingerid:smiles"]
        else:
            return row["csifingerid:smiles"] 
        

    def produce_integrated_graphml(self) -> None:
        """Cleans, filters and creates graphml from network DF and edges
        
        Topology filter is always used: cluster can be only of 100 nodes. 
        Converts all missing values to "N/A" and turns all properties into strings for easier reading in cytoscape!
        """
        integrated_df: pd.DataFrame = self.network_df
        integrated_network = nx.from_pandas_edgelist(self.network_edgelist,edge_attr=True)
        integrated_df_as_strings = integrated_df.astype(str)
        integrated_df_cleaned = integrated_df_as_strings.map(convert_missing)
        nodes_dict = integrated_df_cleaned.to_dict(orient='index')
        for node_ID in nodes_dict:
            if not node_ID in integrated_network:
                integrated_network.add_node(node_ID)
            #nodes_dict[node_ID] = {}
            for attribute in nodes_dict[node_ID]:
                integrated_network.nodes[node_ID][attribute] = nodes_dict[node_ID][attribute]
        filter_component(integrated_network,100)
        nx.write_graphml(integrated_network, f"{self.output_folder}/{self.name}.graphml")

    def run_tool(self,tool: Any) -> None:
        """Get running parameters and run selected tool
        
        Here, all running behavirous for a specific tool is defined. First the required parameters are obtained from properties that were set & checked in `WorkflowSettings`.
        Simple format converions of input are done here as well.  
        No order is defined here: use depth in internal settings for that.
        Will only do something for tools defined in `available_tools` class property.

        """
        match tool:
            case "classyfire":
                #print('ran classyfire (not, debug)')
                input_csv = self.create_smiles_csv_from_df(output_name = f"{self.output_folder}/classyfire_input.csv",header=False)
                output_path = self.settings.paths.get("classyfire_output")
                print('running classyfire')
                run_classyfire(input_path=input_csv,output_path=output_path)
                # parse_classyfire_sdf("/lustre/BIF/nobackup/hendr218/temp/sdftest.txt")
            case "matchms":
                print("running matchms")
                graph: nx.Graph = create_network_from_mgf(self.settings.paths.get("input_mgf"))
                output_path = self.settings.paths.get("base_network")
                graph.export_to_graphml(output_path)
                #nx.write_graphml(graph, output_path)
            case "ms2lda":
                output_path: str = self.settings.paths.get("ms2lda_output")
                self.settings.ms2lda["dataset_parameters"]["output_folder"] = output_path
                ms2lda_params: dict = self.settings.get_ms2lda_params()
                input_mgf: str = self.settings.paths.get("input_mgf")
                #print("ran ms2lda (not, debug)")
                print('running ms2lda')
                run_ms2lda(dataset=input_mgf,params=ms2lda_params)
            case "mzmine":
                mzmine_location = self.settings.paths.get("mzmine_location")
                mzmine_userfile_location = self.settings.paths.get("mzmine_userfile_location")
                base_batchfile = self.settings.paths.get("mzmine_base_batchfile")
                output_loc = f'{self.output_folder}/mzmine'
                #get filelist if it was provided, else fall back to the folder
                self.data: list = self.settings.paths.get("file_list", get_filelist_from_folder(self.settings.paths.get("data_folder")))
                batchfile_writer = mzbatch_writer(base_mzbatch=base_batchfile,data=self.data,mzmine_params=self.settings.mzmine)
                batchfile = batchfile_writer.write_mzbatch(output_filename=f'{output_loc}_mzbatch.mzbatch')
                #print("ran mzmine, not, debug")
                print('running mzmine')
                run_mzmine(batchfn=batchfile,mzmine_output_loc=output_loc,mzmine_location=mzmine_location,mzmine_userfile_location=mzmine_userfile_location,temp_folder=f'{output_loc}/temp/')
            case "sirius":
                output_loc: str = self.settings.paths.get("sirius_output")
                sirius_path: str = self.settings.paths.get("sirius_path")
                input_mgf: str= self.settings.paths.get("input_mgf")
                instrument: str = self.settings.sirius.get("instrument")
                formula_db: str = self.settings.sirius.get("formula_db")
                #print("ran sirius (not, debug)")
                print('running sirius')
                run_sirius(input_path=input_mgf,output_path=output_loc,sirius_path=sirius_path,formula_db=formula_db,instrument=instrument)
            case "toxtree":
                toxtree_loc = self.settings.paths.get("toxtree_path")
                toxtree_module = self.settings.toxtree.get("module")
                config: dict = self.settings.config.get("toxtree",{}).get("running",{}).get("translations",{})
                if not toxtree_module in config.get("modules",{}):
                    raise Settingserror(f"toxtree module {toxtree_module} not supported, add to module translation in internal settings to support")
                toxtree_module = config.get("modules",{}).get(toxtree_module)
                input_csv = self.create_smiles_csv_from_df(output_name = f"{self.output_folder}/toxtree_input.csv",header=True)
                output_path = f"{self.output_folder}/toxtree_results.csv"
                #print("ran toxtree (not, debug)")
                print('running toxtree')
                run_toxtree(input_path=input_csv,output_path=output_path,toxtree_path=toxtree_loc,module_path=toxtree_module)
       
    def integrate_tool(self,tool: Any) -> None:
        """Get needed paramaters and integrate selected tool
        
        Here, all integration behaviour for a specific tool is defined. First the required parameters are obtained from properties that were set & checked in `WorkflowSettings`.
        Simple format converions of input are done here as well.  
        No order is defined here: use depth in internal settings for that.
        Will only do something for tools defined in `available_integrations` class property.

        """
        match tool:
            case "classyfire":
                print('integrating classyfire')
                output_folder: str = self.settings.paths.get("classyfire_output")
                classyfire_records: pd.DataFrame= parse_classyfire_sdf(output_folder)
                config: dict  = self.settings.config.get("classyfire", {}).get("integration", {})
                integration_settings: tuple[dict,str] = get_merging_settings(config,self.integration_col)
                self.network_df: pd.DataFrame = integrate_df_cols_to_df(self.network_df,classyfire_records, *integration_settings)
            case "ms2lda":
                print('integrating ms2lda')
                output_folder = self.settings.paths.get("ms2lda_output")
                dataset = self.settings.paths.get("input_mgf")
                output_database = self.settings.paths.get("ms2lda_results_db")
                self.network_df = integrate_ms2lda_to_df(self.network_df,dataset,output_folder,output_database)
            case "mzmine":
                #temp to get some quick results, will be improved later dataframe: pd.DataFrame,input_mgf: str,metadata_csv: str,quant_table: str
                print('integrating mzmine')
                input_mgf = self.settings.paths.get("sirius_mgf")
                quant_table = self.settings.paths.get("quant_table")
                metadata_csv = self.settings.paths.get("metadata_csv")
                self.network_df = temp_metadata_adder(self.network_df,input_mgf,metadata_csv,quant_table)
            case "sirius":
                print('integrating sirius')
                config: dict = self.settings.config.get("sirius",{}).get("integration",{}).get("translations",{})
                translators = config.get("sirius",{}),config.get("csi:fingerid",{}),config.get("canopus",{})
                outputs = self.settings.paths.get("sirius_tool_output"), self.settings.paths.get("csi:fingerid_output"), self.settings.paths.get("canopus_output")
                self.graph = integrate_sirius_to_graph(graph=self.graph,translators=translators,outputs=outputs)
            case "sirius_db":
                print('integrating sirius from db')
                config: dict = self.settings.config.get("sirius_db",{}).get("integration",{}).get("translations",{})
                translators = config.get("sirius",{}),config.get("csi:fingerid",{}),config.get("canopus",{})
                outputs = self.settings.paths.get("sirius_tool_db_output"), self.settings.paths.get("csi:fingerid_db_output"), self.settings.paths.get("canopus_db_output")
                self.graph = integrate_sirius_to_graph(graph=self.graph,translators=translators,outputs=outputs)
            case "toxtree":
                print('integrating toxtree')
                toxtree_output: str = self.settings.paths.get("toxtree_output")
                toxtree_df: pd.DataFrame = parse_cramer_classifications(toxtree_output)
                config: dict  = self.settings.config.get("toxtree", {}).get("integration", {})
                integration_settings = get_merging_settings(config,self.integration_col)
                self.network_df = integrate_df_cols_to_df(self.network_df,toxtree_df, *integration_settings)
            case "plastchemdb":
                print('integrating plastchem')
                config: dict  = self.settings.config.get("plastchemdb", {}).get("integration", {})
                plastchem_db: str = self.settings.paths.get("plastchem_path")
                plastchem_df =  pd.read_csv(plastchem_db, sep='\t', encoding='windows-1251', low_memory=False, header=1)
                integration_settings = get_merging_settings(config,self.integration_col) 
                self.network_df = integrate_df_cols_to_df(self.network_df,plastchem_df, *integration_settings)
    
    def merge_dfs():
        pass
    def create_smiles_csv_from_df(self,header: bool,output_name: str) -> bool:
        """Creates a smiles file from a dataframe column with rows on newlines"""
        network_df = self.network_df
        #df_smiles = network_df[network_df['csifingerid:smiles'].notna()].rename(columns={"csifingerid:smiles": "smiles"})      
        df_smiles = network_df[network_df['smiles'].notna()]  
        df_smiles.to_csv(output_name,columns=["smiles"],header=header, index=False,na_rep="N/A")
        return output_name 
    
class Settingserror(Exception):
    """Error for indicating invalid settings files"""
    pass

class WorkflowSettings:
    """Class to process and hold settings for the workflow

    First check if the always required fields from `required_settings' and `required_paths` class attributes are present. 
    If so, loads paths and adds new ones depending on the selected tools. (i.e. if a tool creates a file that might be needed downstream, it should add it to paths)
    Then, checks if all tools have the input paths that they need as defined in the internal_settings file
    Finally, tests if 

    Parameters
    ----------
    settings : str
        fn of settings file
    available_tools : list[str]
        tools supported by worklflow: others will be ignored
    available_integrations : list[str]
        integrations supported by worklflow: others will be ignored

    Attributes
    ----------
    required_settings : list[str]
        settings that must be provided in settings json 
    required_settings tools : list[str]
        paths that must be provided in settings settings json paths field

    """
    required_settings: list[str] = ["paths","run_tools","integrate_tools"]
    required_paths: list[str] = ["base_output_folder","internal_settings"]

    def __init__(self,settings_json:str | dict,available_tools: list[str],available_integrations: list[str]) -> None:
        self.input: dict = json_from_string_or_file(settings_json)   
        self.validate_required_settings(settings=self.input)
        self.config: dict = json_from_string_or_file(self.input["paths"]["internal_settings"])

        #Add properties for convenience in use and code
        self.paths: dict = self.input["paths"]
        self.name = self.paths.get("name",ctime().replace(" ","_"))
        self.output_folder = f'{self.paths.get("base_output_folder")}/{self.name}'
        Path(self.output_folder).mkdir(exist_ok=True)
        self.to_run: list[str] = self.select_used_tools(available_tools=available_tools,setting="run_tools")
        self.to_integrate: list[str] = self.select_used_tools(available_tools=available_integrations,setting="integrate_tools")

        #Let tools set additional settings and then check if all requirements are met
        self.set_tool_settings(set(self.to_run+self.to_integrate))
        self.check_tool_requirements(tools=self.to_run,goal="running")
        self.check_tool_requirements(tools=self.to_integrate,goal="integration")

        self.check_tool_requirements(tools=["all_tools"],goal="running")
        self.check_tool_requirements(tools=["all_tools"],goal="integration")


    def validate_required_settings(self,settings: dict) -> None:
        """Check if settings json contains required fields"""
        if not validate_dictkeys(settings,WorkflowSettings.required_settings):
            raise Settingserror(f"settings json must contain {', '.join(WorkflowSettings.required_settings)}")
        if not validate_dictkeys(settings["paths"],WorkflowSettings.required_paths):
            raise Settingserror(f"settings json paths must contain {', '.join(WorkflowSettings.required_paths)}")
    

    def check_tool_requirements(self,tools: Iterable[str],goal: str) -> None:
        for tool in tools:
            requirements = self.get_tool_requirements(tool,goal)
            self.validate_tool_requirements(tool,requirements)               
    
    def set_tool_settings(self,tools: Iterable[str]) -> None:
        """set tool settings as property and add any settings that depend on the tool""" 
        for tool in tools:
            tool_settings = self.input.get(tool,{})
            match tool:
                case "mzmine":
                    self.mzmine = tool_settings
                    self.paths["input_mgf"] = f'{self.output_folder}/mzmine/mzmine_iimn_fbmn.mgf'
                    self.paths["sirius_mgf"] = f'{self.output_folder}/mzmine/mzmine_sirius.mgf'
                    self.paths["quant_table"] = f'{self.output_folder}/mzmine/mzmine_iimn_fbmn_quant_full.csv'
                case "sirius":
                    self.sirius = tool_settings
                    sirius_loc = f'{self.output_folder}/sirius/'
                    Path(sirius_loc).mkdir(exist_ok=True)
                    self.paths["canopus_output"] = f"{sirius_loc}canopus_structure_summary.tsv"
                    self.paths["csi:fingerid_output"] = f"{sirius_loc}structure_identifications.tsv"
                    self.paths["sirius_tool_output"] = f"{sirius_loc}formula_identifications.tsv"
                    self.paths["sirius_output"] = sirius_loc
                case "sirius_db":
                    self.siriusdb = tool_settings
                    siriusdb_loc = f'{self.output_folder}/sirius_db/'
                    self.paths["canopus_db_output"] = f"{siriusdb_loc}canopus_structure_summary.tsv"
                    self.paths["csi:fingerid_db_output"] = f"{siriusdb_loc}structure_identifications.tsv"
                    self.paths["sirius_tool_db_output"] = f"{siriusdb_loc}formula_identifications.tsv"
                    self.paths["sirius_db_output"] = siriusdb_loc
                case "matchms":
                    self.matchms = tool_settings
                    self.paths["base_network"] = f'{self.output_folder}/base_network.graphml'
                case "toxtree":
                    self.toxtree = tool_settings
                    self.paths["toxtree_output"] = f"{self.output_folder}/toxtree_results.csv"
                case "classyfire":
                    self.classyfire = tool_settings
                    self.paths["classyfire_output"] = f"{self.output_folder}/classyfire_results.sdf"
                case "ms2lda":
                    self.ms2lda = tool_settings
                    ms2lda_loc = f"{self.output_folder}/ms2lda"
                    self.paths["ms2lda_output"] = f"{ms2lda_loc}"
                    self.paths["ms2lda_results_db"] = f"{self.output_folder}/ms2lda/motifDB_optimized.xlsx"

    def get_tool_requirements(self,tool: str, goal:str) -> tuple[list,list,list,list]:
        """Get requirements if available, otherwise return emoty list"""
        config = self.config.get(tool,{}).get(goal,{}).get("requirements",{})
        required_paths: Iterable[Any] = config.get("paths",[])
        required_settings: Iterable[Any] = config.get("settings",[])
        required_optional_paths: Iterable[Iterable[Any]] = config.get("optional_paths",[])
        required_optional_settings: Iterable[Iterable[Any]] = config.get("optional_settings",[])
        return required_paths,required_settings,required_optional_paths,required_optional_settings
    
    def validate_tool_requirements(self,tool, requirements):
        """Raise errors if required settings are not present in settings"""
        required_paths,required_settings,required_optional_paths,required_optional_settings = requirements
            # if not validate_dictkeys(self.input["paths"],required_paths):
        if not validate_dictkeys(self.paths,required_paths):
            raise Settingserror(f"all of {required_paths} required to run {tool}, but one is missing in paths settings")
        if not validate_dictkeys(self.input.get(tool),required_settings):
            raise Settingserror(f"One of {required_settings} missing in {tool} settings")
        for combination in required_optional_paths:
            if not validate_dictkeys(self.paths,combination):
                    raise Settingserror(f"Any one of {combination} required to run {tool}, but one is missing in paths settings")
        for combination in required_optional_settings:
            if not validate_dictkeys(self.input.get(tool),combination):
                    raise Settingserror(f"Any of {combination} is required in {tool} settings")
# test2 = [self.network_df[self.network_df["smiles"].notna()].merge(test[test["canonical_smiles"].notna()],)
# target_df[target_df["Molecular formula"].notna()].merge(source_df[source_df["molecular_formula"].notna()],left_on="smiles",right_on="molecular_formula")
    def select_used_tools(self,available_tools: list,setting: str) -> list:
        """Check which available tools were selected in settings file"""
        return [tool for tool in available_tools if self.input[setting].get(tool) == "True"]
    
    def get_ms2lda_params(self):
        """Get the parameters ms2lda needs froms settings"""
        preprocessing_parameters = self.ms2lda["preprocessing_parameters"]
        convergence_parameters = self.ms2lda["convergence_parameters"]
        annotation_parameters = self.ms2lda["annotation_parameters"]
        model_parameters = self.ms2lda["model_parameters"]
        train_parameters = self.ms2lda["train_parameters"]
        fingerprint_parameters = self.ms2lda["fingerprint_parameters"]
        dataset_parameters = self.ms2lda["dataset_parameters"]
        n_motifs = self.ms2lda["n_motifs"]
        n_iterations = self.ms2lda["n_iterations"]
        motif_parameter = self.ms2lda["motif_parameter"]

        return preprocessing_parameters,convergence_parameters,annotation_parameters,model_parameters,train_parameters,fingerprint_parameters,dataset_parameters,n_iterations,n_motifs,motif_parameter


In [ ]:
from main import WorkflowRunner,WorkflowSettings


In [ ]:
with open("/lustre/BIF/nobackup/hendr218/Data/with_pcdb_copy/first_test/first_test.pickle","rb") as file:
    wf = pickle.load(file)

In [ ]:
with open("/lustre/BIF/nobackup/hendr218/Data/with_pcdb_copy/first_test/first_test.pickle","rb") as file:
    wf = pickle.load(file)
df = wf.network_df
nw = df

from typing import Iterable
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import main


In [ ]:
# def create_vc_df(df,col_to_plot,col_to_filter=None):
#     if col_to_filter:
#         df = df.loc[:,[col_to_plot]][(df[col_to_filter] == True) & (df["is_blank"] == False)]
#     else:
#          df = df.loc[:,[col_to_plot]][df["is_blank"] == False]
#     df = df.value_counts(normalize=True)
#     df = df.to_dict()
#     df = pd.DataFrame.from_dict(df,orient="index",columns=[col_to_plot])
#     return df

In [ ]:
from typing import Iterable
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import main


In [ ]:

def create_counts_df(df: pd.DataFrame,col_to_plot: str,col_to_filter:str=None,normalize:bool=True) -> pd.DataFrame:
    """Creates a dataframe for"""
    df = df[df[col_to_plot]!= "N/A"] 
    if col_to_filter:
        df = df[col_to_plot][(df[col_to_filter] == True) & (df["is_blank"] == False)].value_counts(normalize=normalize)
    else: 
        df = df[col_to_plot][df["is_blank"] == False].value_counts(normalize=normalize)
    df=df.to_dict()
    df = pd.DataFrame.from_dict(df,orient="index")
    if col_to_filter is None:
        return df.rename(columns={0:"All"})
    return df.rename(columns={0:col_to_filter})

def join_counts(*dataframes: Iterable[pd.DataFrame]):
    if len(dataframes) == 1:
        return dataframes
    dfs = [dataframe for dataframe in dataframes]
    df_joined = dfs.pop(0)
    for i,df in enumerate(dfs):
        df_joined = df_joined.join(df,how="outer") #lsuffix=f"_{i+1}",rsuffix=f"_{i+2}"
    return df_joined.fillna(0)

def df_to_barplot(df:pd.DataFrame,columns:list[str],title: str,ytitle:str,fn: str) -> None:
    ax = df[columns].plot(kind='bar', title=title, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel(ytitle, fontsize=12)
    plt.tight_layout()
    plt.savefig(fn)


def df_to_counts_plot(df:pd.DataFrame,filter_cols: list[str],vis_col: str,fn:str,title:str,relative_counts: bool = True,include_nofilter:bool=False) -> None:
    count_dfs = [create_counts_df(df,vis_col,filter_col,normalize=relative_counts) for filter_col in filter_cols]
    if include_nofilter:
        count_dfs += create_counts_df(df,vis_col,normalize=relative_counts)
        filter_cols += "All"
    joined_df = join_counts(*count_dfs)
    ytitle = "Feature count"
    if relative_counts:
        joined_df = joined_df.apply(lambda x: x*100)
        ytitle = "occurence (%)"
    df_to_barplot(joined_df,columns=filter_cols,title=title,ytitle=ytitle,fn=fn)

In [ ]:
# visualization_cols = ["CF:subclass", "CF:class", "CF:superclass"]
# vis_dfs = [create_counts_df(visualization_col) for visualization_col in visualization_cols]

def plot_classes(loc,relative_counts=True,filter_type="type",class_type="all"):
    filter_dict = {
        "type":["PE","PE_PET","PE_PA"],
        "state":["Removed_by_dec","Introduced_by_dec","Kept_by_dec"]
    } 
    class_dict={
        "canopus":["canopus:CF_subclass", "canopus:CF_class", "canopus:CF_superclass"],
        "classyfire":["CF:subclass", "CF:class", "CF:superclass"],
        "all":["subclass", "class", "superclass"]
    }
    vis_cols = class_dict[class_type]
    filter_cols = filter_dict[filter_type]

    counttype = "relative" if relative_counts else "absolute"
    for vis_col in vis_cols:
        
        fn = f"{loc}/{class_type}_classes_by_{filter_type}_{vis_col}_{counttype}_counts.png"
        df_to_counts_plot(nw,filter_cols,vis_col,fn=fn,title=vis_col,relative_counts=relative_counts)
        fn = f"{fn[:-4]}_all.png"
        df_to_counts_plot(nw,filter_cols,vis_col,fn=fn,title=vis_col,relative_counts=relative_counts,include_nofilter=True)

# settings =     fn="/lustre/BIF/nobackup/hendr218/Data/test.png" title="testtitle",relative_counts=False,ytitle="Feature_count"






In [ ]:
test = ["PE","PE_PET","PE_PA"]
test.append(None)
print(test)

In [ ]:
plt.close()

In [ ]:
loc = "/lustre/BIF/nobackup/hendr218/Data/plot_tests"
for filter_type in ["type","state"]:
    for class_type in ["canopus","classyfire","all"]:
        plot_classes(loc,relative_counts=True,filter_type=filter_type,class_type=class_type)
        plot_classes(loc,relative_counts=False,filter_type=filter_type,class_type=class_type)
        plt.close()

In [ ]:
dataframes = count_dfs
dfs = [dataframe for dataframe in dataframes]
df_joined = dfs.pop(0)
# dfs.pop(0)
# for i,df in enumerate(dfs):
#     df_joined = df_joined.join(df,lsuffix=f"_{i-1}",rsuffix=f"_{i}",how="outer")
# df_joined.fillna(0)
# # return df_joined

In [ ]:
df_joined = df_joined.join(dfs[1],lsuffix="_left",rsuffix="_right",how="outer")

In [ ]:
df_joined.fillna(0)

In [ ]:
def create_vc_df(df,col_to_plot,col_to_filter=None):
    if col_to_filter:
        df = df[col_to_plot][(df[col_to_filter] == True) & (df["is_blank"] == False)].value_counts(normalize=True)
    else: 
        df = df[col_to_plot][df["is_blank"] == False].value_counts(normalize=True)
    df=df.to_dict()
    df = pd.DataFrame.from_dict(df,orient="index",columns=[col_to_plot])
    return df

In [ ]:
def create_vc_df(df,col_to_plot,col_to_filter=None,normalize=True):
    if col_to_filter:
        df = df[col_to_plot][(df[col_to_filter] == True) & (df["is_blank"] == False)].value_counts(normalize=normalize)
    else: 
        df = df[col_to_plot][df["is_blank"] == False].value_counts(normalize=normalize)
    df=df.to_dict()
    df = pd.DataFrame.from_dict(df,orient="index",columns=[col_to_plot])
    return df

In [ ]:
col_to_plot = "canopus:CF_superclass"
col_to_filter = "PE_PET"
nw = df

In [ ]:
tall = nw["canopus:CF_superclass"][(nw["PE_PET"] == True) & (nw["is_blank"] == False)].value_counts(normalize=True)
tall=tall.to_dict()
tall = pd.DataFrame.from_dict(tall,orient="index",columns=[col_to_plot])
tall

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df["all_metadata"]

In [ ]:
onw = nw

In [ ]:
for vis_col in ["CF:subclass", "CF:class", "CF:superclass"]:
    nw = onw[onw[vis_col] != "N/A"]
    PE_PET = create_vc_df(nw,vis_col,col_to_filter="PE_PET")
    PE_PA = create_vc_df(nw,vis_col,col_to_filter="PE_PA")
    PE = create_vc_df(nw,vis_col,col_to_filter="PE")
    ALL = create_vc_df(nw,vis_col)

    df_joined = PE.join(PE_PET,lsuffix="_PE",rsuffix="_PET", how="outer")
    df_joined2 = ALL.join(PE_PA,lsuffix="_ALL",rsuffix="_PA", how="outer")
    df_joined = df_joined2.join(df_joined,how="outer").fillna(0)
    df_joined = df_joined.apply(lambda x: x*100)
    df_joined = df_joined.rename(columns=lambda x: x.split("_")[-1])
    if vis_col == "subclass":
        df_joined = df_joined[df_joined.apply(lambda row: any([i > 1  for i in row]),axis=1)] 
    ax = df_joined[["ALL","PA","PE","PET"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_{vis_col}_by_plastic_type.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
for vis_col in ["subclass", "class", "superclass"]:
    PE_PET = create_vc_df(nw,vis_col,col_to_filter="PE_PET")
    PE_PA = create_vc_df(nw,vis_col,col_to_filter="PE_PA")
    PE = create_vc_df(nw,vis_col,col_to_filter="PE")
    ALL = create_vc_df(nw,vis_col)

    df_joined = PE.join(PE_PET,lsuffix="_PE",rsuffix="_PET", how="outer")
    df_joined2 = ALL.join(PE_PA,lsuffix="_ALL",rsuffix="_PA", how="outer")
    df_joined = df_joined2.join(df_joined,how="outer").fillna(0)
    df_joined = df_joined.apply(lambda x: x*100)
    df_joined = df_joined.rename(columns=lambda x: x.split("_")[-1])
    if vis_col == "subclass":
        df_joined = df_joined[df_joined.apply(lambda row: any([i > 1  for i in row]),axis=1)] 
    ax = df_joined[["ALL","PA","PE","PET"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_{vis_col}_by_plastic_type.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
for vis_col in ["subclass", "class", "superclass"]:
    PE_PET = create_vc_df(nw,vis_col,col_to_filter="PE_PET")
    PE_PA = create_vc_df(nw,vis_col,col_to_filter="PE_PA")
    PE = create_vc_df(nw,vis_col,col_to_filter="PE")
    ALL = create_vc_df(nw,vis_col)

    df_joined = PE.join(PE_PET,lsuffix="_PE",rsuffix="_PET", how="outer")
    df_joined2 = ALL.join(PE_PA,lsuffix="_ALL",rsuffix="_PA", how="outer")
    df_joined = df_joined2.join(df_joined,how="outer").fillna(0)
    df_joined = df_joined.apply(lambda x: x*100)
    df_joined = df_joined.rename(columns=lambda x: x.split("_")[-1])
    if vis_col == "subclass":
        df_joined = df_joined[df_joined.apply(lambda row: any([i > 1  for i in row]),axis=1)] 
    ax = df_joined[["ALL","PA","PE","PET"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_{vis_col}_by_plastic_type.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
for vis_col in ["CF:subclass", "CF:class", "CF:superclass"]:
    nw = onw[onw[vis_col] != "N/A"]
    DEC = create_vc_df(nw,vis_col,col_to_filter="Decontaminated")
    PC = create_vc_df(nw,vis_col,col_to_filter="Postconsumer")
    ALL = create_vc_df(nw,vis_col)

    df_joined = DEC.join(PC,lsuffix="_DEC",rsuffix="_PC", how="outer")
    df_joined2 = ALL.join(df_joined,how="outer",lsuffix="_ALL")
    df_joined = df_joined2.rename(columns={vis_col:"ALL",f"{vis_col}_DEC": "Decontaminated",f"{vis_col}_PC":"Postconsumer"})

    df_joined = df_joined.apply(lambda x: x*100)
    ax = df_joined[["ALL","Decontaminated","Postconsumer"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_{vis_col}_by_plastic_state.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
df["removed_by_dec"]

In [ ]:
for vis_col in ["CF:subclass", "CF:class", "CF:superclass"]:
    nw = onw[onw[vis_col] != "N/A"]
    RM = create_vc_df(nw,vis_col,col_to_filter="removed_by_dec")
    IN = create_vc_df(nw,vis_col,col_to_filter="Introduced_by_dec")
    ALL = create_vc_df(nw,vis_col)

    df_joined = RM.join(IN,lsuffix="_RM",rsuffix="_IN", how="outer")
    df_joined2 = ALL.join(df_joined,how="outer",lsuffix="_ALL")
    df_joined = df_joined2.rename(columns={vis_col:"ALL",f"{vis_col}_RM": "Removed",f"{vis_col}_IN":"Introduced"})
    df_joined = df_joined.apply(lambda x: x*100)
    ax = df_joined[["ALL","Removed","Introduced"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_{vis_col}_intod_remov.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
df[df["Kept_by_dec"]]


In [ ]:
create_vc_df(nw,vis_col,col_to_filter="Removed_by_dec",normalize=False)

In [ ]:
def create_vc_df(df,col_to_plot,col_to_filter=None,normalize=True):
    if col_to_filter:
        df = df[col_to_plot][(df[col_to_filter] == True) & (df["is_blank"] == False)].value_counts(normalize=normalize)
    else: 
        df = df[col_to_plot][df["is_blank"] == False].value_counts(normalize=normalize)
    df=df.to_dict()
    df = pd.DataFrame.from_dict(df,orient="index",columns=[col_to_plot])
    return df

In [ ]:
df[col_to_filter]

In [ ]:
col_to_filter="Removed_by_dec"
nw[col_to_filter]

In [ ]:
for vis_col in ["CF:subclass", "CF:class", "CF:superclass"]:
    nw = onw[onw[vis_col] != "N/A"]
    RM = create_vc_df(nw,vis_col,col_to_filter="Removed_by_dec",normalize=False)
    IN = create_vc_df(nw,vis_col,col_to_filter="Introduced_by_dec",normalize=False)
    KP = create_vc_df(nw,vis_col,col_to_filter="Kept_by_dec",normalize=False)
    ALL = create_vc_df(nw,vis_col,normalize=False)
    df_joined = RM.join(IN,lsuffix="_Removed",rsuffix="_Introduced", how="outer")
    df_joined2 = ALL.join(KP,how="outer",lsuffix="_ALL",rsuffix="_Kept")
    df_joined = df_joined2.join(df_joined,how="outer").fillna(0)
    df_joined = df_joined.rename(columns=lambda x: x.split("_")[-1])
    # df_joined = df_joined.apply(lambda x: x*100)
    ax = df_joined[["ALL","Removed","Introduced","Kept"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Amount of features", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_counts_{vis_col}_intod_remov_kept.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
print(len([0,1]))

In [ ]:
for vis_col in ["CF:subclass", "CF:class", "CF:superclass"]:
    nw = onw[onw[vis_col] != "N/A"]
    RM = create_vc_df(nw,vis_col,col_to_filter="Removed_by_dec",normalize=True)
    IN = create_vc_df(nw,vis_col,col_to_filter="Introduced_by_dec",normalize=True)
    KP = create_vc_df(nw,vis_col,col_to_filter="Kept_by_dec",normalize=True)
    ALL = create_vc_df(nw,vis_col,normalize=True)
    df_joined = RM.join(IN,lsuffix="_Removed",rsuffix="_Introduced", how="outer")
    df_joined2 = ALL.join(KP,how="outer",lsuffix="_ALL",rsuffix="_Kept")
    df_joined = df_joined2.join(df_joined,how="outer").fillna(0)
    df_joined = df_joined.rename(columns=lambda x: x.split("_")[-1])
    df_joined = df_joined.apply(lambda x: x*100)
    ax = df_joined[["ALL","Removed","Introduced","Kept"]].plot(kind='bar', title =vis_col, figsize=(15, 10), legend=True, fontsize=12)
    ax.set_ylabel("Occurence (%)", fontsize=12)
    fig_name=f"/lustre/BIF/nobackup/hendr218/Data/cf_only_occurence_{vis_col}_intod_remov_kept.png"
    plt.tight_layout()
    plt.savefig(fig_name)

In [ ]:
def filter_component(G, max_component_size):
    """From GNPS! drop lowest edges from cluster if it exceeds max size """
    if max_component_size == 0:
        return

    big_components_present = True

    while big_components_present == True:
        big_components_present = False
        components = nx.connected_components(G)
        for component in components:
            if len(component) > max_component_size:
                prune_component(G, component)
                big_components_present = True
        #print("After Round of Component Pruning", len(G.edges()))
        
def prune_component(G, component, cosine_delta=0.02):
    component_edges = get_edges_of_component(G, component)

    min_score = 1000
    for edge in component_edges:
        min_score = min(min_score, edge[2]["weight"])

    cosine_threshold = cosine_delta + min_score
    for edge in component_edges:
        if edge[2]["weight"] < cosine_threshold:
            #print(edge)
            G.remove_edge(edge[0], edge[1])

def get_edges_of_component(G, component):
    component_edges = {}
    for node in component:
        node_edges = G.edges((node), data=True)
        for edge in node_edges:
            if edge[0] < edge[1]:
                key = edge[0] + "-" + edge[1]
            else:
                key = edge[1] + "-" + edge[0]
            component_edges[key] = edge

    component_edges = component_edges.values()
    return component_edges

#from 
def batched(iterable, n, *, strict=False):
    """from itertools python 3.12, split iterable in batches of size n"""
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch



def convert_missing(x):
    """Helper to convert missing value to the same format"""
    if (x=="nan") or (x.lower() == "n/a"):
        return "N/A"
    else:
        return x

In [ ]:
from typing import Iterable
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import main
import itertools
import networkx as nx
with open("/lustre/BIF/nobackup/hendr218/Data/with_pcdb_copy/first_test/first_test.pickle","rb") as file:
    wf = pickle.load(file)
df = wf.network_df
nw = df




In [ ]:
test = wf.network_df["MS2LDA:allmotifs"]
test = test[test != "N/A"]

In [ ]:
def get_features_per_motif(motif_col: pd.DataFrame) -> dict[str,list[str]]:
    features_per_motif= {}
    for feature_id, motifs in test.to_dict().items():
        for motif in motifs:
            if not motif in features_per_motif:
                features_per_motif[motif] = []
                features_per_motif[motif].append(feature_id)
            if not feature_id in features_per_motif[motif]:
                features_per_motif[motif].append(feature_id)
    return features_per_motif

In [ ]:
fpm = get_features_per_motif(test)

In [ ]:
fpm_short = {motif:features for motif,features in fpm.items() if len(features) < 100}

In [ ]:
len(fpm_short)

In [ ]:
new_edges = []
# G = nx.MultiGraph()
G=nx.from_pandas_edgelist(wf.network_edgelist,edge_attr=True,create_using=nx.MultiGraph)
for edge in G.edges:
    G.edges[edge]["type"]="cosine"
filter_component(G,100)
# # H.edges["type"]="cosine"
# # print(len(H.edges))

integrated_df: pd.DataFrame = wf.network_df
integrated_df_as_strings = integrated_df.astype(str)
integrated_df_cleaned = integrated_df_as_strings.map(convert_missing)
nodes_dict = integrated_df_cleaned.to_dict(orient='index')
for node_ID in nodes_dict:
    if not node_ID in G:
        G.add_node(node_ID)
            #nodes_dict[node_ID] = {}
        for attribute in nodes_dict[node_ID]:
            G.nodes[node_ID][attribute] = nodes_dict[node_ID][attribute]
        # filter_component(G,100)

for key in fpm_short:
    test = key
    combinations = list(itertools.combinations(fpm_short[key],2))
    # print(combinations)
    combinations = [tuple([combination[0],combination[1],{key:True}]) for combination in combinations]
    G.add_edges_from(combinations,motif=key,type="motif")
#     # G.add_edges_from(H.edges,type="cosine")

nx.write_graphml(G,"/lustre/BIF/nobackup/hendr218/Data/post_motif_edges.graphml")
#     # nx.set_edge_attributes(G,combinations,key)
#     # for i,j in itertools.combinations(fpm[key],2):
#     #     test.add_edge(i,j,key=True)

In [ ]:
print([len(fpm[motif]) for motif in fpm if len(fpm[motif]) > 100])
print(len(fpm))

In [ ]:
nx.from_pandas_edgelist(wf.network_edgelist,edge_attr=True,create_using=nx.MultiGraph).edges



In [ ]:
A = nx.MultiGraph()
A.add_edges_from([(1,2),(2,3),(4,5),(5,7),(5,6),(4,7)],shape="triangle",color="red",edgetype="cosine")
A.add_edges_from([(1,5),(2,7)],motif="motif_121",shape="square",edgetype="motif")
A.add_edges_from([(2,6),(1,4),(4,7)],motif="motif_300",shape="square",edgetype="motif")

In [ ]:
nx.write_graphml(A,"/lustre/BIF/nobackup/hendr218/Data/multigraph_test.graphml")

In [ ]:
edge1 edge2 edgetype label properties

In [36]:
import pickle
from matchms.importing import scores_from_pickle
from src.myworkflow.running import *
from src.myworkflow.integrating import *

In [125]:

def create_network_from_scores(scores: matchms.Scores ) -> nx.Graph:
    """Filters scores and creates network"""
    #scores: matchms.Scores = scores_from_pickle("/lustre/BIF/nobackup/hendr218/temp/scores_001")
    newscores = scores
    newscores.filter_by_range(name="ModifiedCosine_matches",low=6,below_operator="<")
    if "scans" in newscores.queries[0].metadata:
        ms_network = SimilarityNetwork(identifier_key="scans",score_cutoff=0.7,top_n=10,link_method="mutual")
    elif "feature_id" in newscores.queries[0].metadata:
         ms_network = SimilarityNetwork(identifier_key="feature_id",score_cutoff=0.7,top_n=10,link_method="mutual")
    else:
        raise ValueError("'Scans' or 'feature_id' required in mgf to identify features in network by other tools")
    ms_network.create_network(newscores,score_name="ModifiedCosine_score")
    return ms_network

In [37]:
a = scores_from_pickle("/lustre/BIF/nobackup/hendr218/temp/scores_V1.pickle")

print(type(a))

<class 'matchms.Scores.Scores'>


In [40]:
b = create_network_from_scores(a)

/lustre/BIF/nobackup/hendr218/miniforge3/envs/workflow_V5/lib/python3.11/site-packages/matchms/networking/SimilarityNetwork.py:131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (ref_candidates != query_id))[0][:self.max_links]


In [46]:
ms_network = SimilarityNetwork(identifier_key="test",score_cutoff=0.7,top_n=10,link_method="mutual")
c = ms_network.create_network(a,score_name="ModifiedCosine_score")


ValueError: None cannot be a node

In [112]:
for query in b.queries:
    query.metadataquery.metadata.pop("feature_id")

AttributeError: 'Spectrum' object has no attribute 'metadataquery'

In [110]:
b.queries[0].metadata = {key: val for key,val in b.queries[0].metadata.items() if key != "scans"}

In [111]:
b.queries[0].metadata

{'charge': 1,
 'retention_time': 2125.484,
 'ms_level': '2',
 'precursor_mz': 90.5069}

In [97]:
for i, item in enumerate(b.queries):
    print(item)

Spectrum(precursor m/z=90.51, 5 fragments between 44.0 and 154.7)
Spectrum(precursor m/z=91.50, 3 fragments between 55.3 and 154.7)
Spectrum(precursor m/z=97.06, 3 fragments between 54.0 and 67.1)
Spectrum(precursor m/z=97.06, 3 fragments between 55.3 and 67.1)
Spectrum(precursor m/z=99.51, 4 fragments between 55.3 and 154.7)
Spectrum(precursor m/z=106.07, 5 fragments between 51.0 and 79.1)
Spectrum(precursor m/z=106.07, 3 fragments between 55.3 and 79.1)
Spectrum(precursor m/z=107.05, 4 fragments between 51.0 and 79.1)
Spectrum(precursor m/z=107.05, 5 fragments between 51.0 and 79.1)
Spectrum(precursor m/z=107.07, 4 fragments between 45.0 and 55.3)
Spectrum(precursor m/z=107.07, 1 fragments between 45.0 and 45.0)
Spectrum(precursor m/z=107.07, 2 fragments between 45.0 and 55.3)
Spectrum(precursor m/z=111.01, 1 fragments between 69.0 and 69.0)
Spectrum(precursor m/z=111.02, 9 fragments between 42.0 and 154.7)
Spectrum(precursor m/z=111.02, 6 fragments between 42.0 and 88.5)
Spectrum(pr

In [104]:
b.queries[0].metadata["test"] = 5
print(b.queries[0].metadata)

{'scans': '39651', 'charge': 1, 'retention_time': 2125.484, 'ms_level': '2', 'precursor_mz': 90.5069}


In [106]:
for i, item in enumerate(b.queries):
      query.metadata.pop("scans")

In [107]:
b.queries[0].metadata

{'scans': '39651',
 'charge': 1,
 'retention_time': 2125.484,
 'ms_level': '2',
 'precursor_mz': 90.5069}

In [123]:
for i, item in  enumerate(b.queries):
            item.metadata["scans"] =  i
            print(item.metadata)

{'charge': 1, 'retention_time': 2125.484, 'ms_level': '2', 'precursor_mz': 90.5069}
{'feature_id': '39608', 'scans': '39608', 'charge': 1, 'retention_time': 2125.159, 'ms_level': '2', 'precursor_mz': 91.5047}
{'feature_id': '1084', 'scans': '1084', 'charge': 1, 'retention_time': 509.872, 'ms_level': '2', 'precursor_mz': 97.0648}
{'feature_id': '1451', 'scans': '1451', 'charge': 1, 'retention_time': 546.616, 'ms_level': '2', 'precursor_mz': 97.0648}
{'feature_id': '39442', 'scans': '39442', 'charge': 1, 'retention_time': 2125.984, 'ms_level': '2', 'precursor_mz': 99.5122}
{'feature_id': '48', 'scans': '48', 'charge': 1, 'retention_time': 271.593, 'ms_level': '2', 'precursor_mz': 106.0651}
{'feature_id': '147', 'scans': '147', 'charge': 1, 'retention_time': 357.639, 'ms_level': '2', 'precursor_mz': 106.0652}
{'feature_id': '39389', 'scans': '39389', 'charge': 1, 'retention_time': 2134.927, 'ms_level': '2', 'precursor_mz': 107.0491}
{'feature_id': '14884', 'scans': '14884', 'charge': 1, '

In [120]:
b.queries[0].metadata

{'charge': 1,
 'retention_time': 2125.484,
 'ms_level': '2',
 'precursor_mz': 90.5069}

In [127]:
create_network_from_scores(a)

ValueError: 'Scans' or 'feature_id' required in mgf to identify features in network by other tools

In [130]:
test = [1,2,3]
test2 = [2,3,4,5,6]
Any([item for item in test item in test2])

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (2988147846.py, line 3)

In [ ]:
print('hi')